In [6]:
%matplotlib inline
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply, Embedding, Dropout, BatchNormalization
from keras.layers import RepeatVector, Dense, Activation, Lambda, Flatten, Reshape
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras
import keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import random
import codecs

from pythainlp.tokenize import word_tokenize

In [7]:
from os import listdir
from os.path import isfile, join

In [8]:
sunthornphu_path = listdir("db/sunthornphu/")
other_path = listdir("db/other/")

In [9]:
data_sunthornphu = []
for i in sunthornphu_path:
    with codecs.open("db/sunthornphu/"+str(i), "r", encoding="utf-8") as f:
        data_sunthornphu.extend(f.readlines())
    f.close()
    
for i in range(len(data_sunthornphu)):
    data_sunthornphu[i] = data_sunthornphu[i][0:-2]

In [10]:
data_other = []
for i in other_path:
    with codecs.open("db/other/"+str(i), "r", encoding="utf-8") as f:
        data_other.extend(f.readlines())
    f.close()
    
for i in range(len(data_other)):
    data_other[i] = data_other[i][0:-2]

In [11]:
### Add this
for i in range(1,len(data_sunthornphu)//len(data_other)):
    data_sunthornphu = data_sunthornphu+data_other

In [12]:
########## Character Level #############

word_to_idx = {"<PAD>":0, "<s>":1, "</s>":2}
idx_to_word = {0:"<PAD>", 1:"<s>", 2:"</s>"}

i = 3
maxlen = 0
for sentence_poem in data_sunthornphu:
    if(len(sentence_poem) > maxlen):
        maxlen = len(sentence_poem)
    for word_poem in sentence_poem:
        if(word_poem == "\r" or word_poem == "\n"):
            break
        if(word_poem not in word_to_idx.keys()):
            word_to_idx[word_poem] = i
            idx_to_word[i] = word_poem
            i+=1

vocab_size = len(idx_to_word)
##############################################

########## Word Level #############
# word_to_idx = {"<PAD>":0, "<s>":1, "</s>":2, "UNK":3}
# idx_to_word = {0:"<PAD>", 1:"<s>", 2:"</s>", 3:"UNK"}
# maxlen = 0
# ########## Word Count ############
# word_dict_count = {}
# for i in range(len(data_sunthornphu)):
#     tmp = []
#     for j in data_sunthornphu[i].split("/"):
#         tmp.extend(word_tokenize(j, engine="newmm", whitespaces=False))
#         tmp.extend("/")
#     data_sunthornphu[i] = tmp[0:-1]
#     if( len(data_sunthornphu[i]) > maxlen):
#         maxlen = len(data_sunthornphu[i])
#     for word_poem in data_sunthornphu[i]:
#         if(word_poem not in word_dict_count.keys()):
#             word_dict_count[word_poem] = 1
#         else:
#             word_dict_count[word_poem]+=1
# print("Finished Counting")
            
# word_count = 3
# for i in range(len(data_sunthornphu)):
#     for word_poem in data_sunthornphu[i]:
#         if((word_poem not in word_to_idx.keys()) and word_dict_count[word_poem]>5):
#             word_to_idx[word_poem] = word_count
#             idx_to_word[word_count] = word_poem
#             word_count+=1
            
# vocab_size = len(idx_to_word)
##############################################

In [13]:
# pd.Series(list(word_dict_count.values())).value_counts()
vocab_size

99

In [14]:
X = []
Y = []
data_size = len(data_sunthornphu)

################# Character Level ################
for sentence_poem in data_sunthornphu:
    tmp_X = [1]
    tmp_Y = []
    for char_poem in sentence_poem:
        tmp_X.append(word_to_idx[char_poem])
        tmp_Y.append(word_to_idx[char_poem])
    X.append(tmp_X)
    tmp_Y.append(2)
    Y.append(tmp_Y)
###################################################

################# Word Level ################
# for sentence_poem in data_sunthornphu:
#     tmp_X = [1]
#     tmp_Y = [1]
#     for word_poem in sentence_poem:
#         if(word_dict_count[word_poem]<=5):
#             tmp_X.append(word_to_idx['UNK'])
#             tmp_Y.append(word_to_idx['UNK'])
#             continue
#         tmp_X.append(word_to_idx[word_poem])
#         tmp_Y.append(word_to_idx[word_poem])
#     tmp_X.append(2)
#     tmp_Y.append(2)
#     X.append(tmp_X)
#     Y.append(tmp_Y)
###################################################

# X = pad_sequences(np.array(X[0:20000]), maxlen=maxlen)
# # X = to_categorical(X,vocab_size)
# # X = X.reshape(10000, maxlen , vocab_size)

# Y = pad_sequences(np.array(Y[0:20000]), maxlen=maxlen)
# Y = to_categorical(Y,vocab_size)
# Y = Y.reshape(20000, maxlen , vocab_size)

# print("X shape : ",X.shape)
# print("Y shape : ",Y.shape)

In [15]:
# from keras.activations import softmax
# def softMaxAxis1(x):
#     return softmax(x,axis=1)

# Tx = X.shape[1]
# Ty = Y.shape[1]
# # Ty = 40
# m = data_size

# #These are global variables (shared layers)
# repeator = RepeatVector(Tx)
# concatenator = Concatenate(axis=-1)
# #Attention function###
# fattn_1 = Dense(256, activation = "tanh")
# fattn_2 = Dense(16, activation = "relu")
# ###
# activator = Activation(softMaxAxis1, name='attention_scores') 
# dotor = Dot(axes = 1)

# def one_step_attention(a, s_prev):

#     # Repeat the decoder hidden state to concat with encoder hidden states
#     s_prev = repeator(s_prev)
#     concat = concatenator([a,s_prev])
#     # attention function
#     e = fattn_1(concat)
#     energies =fattn_2(e)
#     # calculate attention_scores (softmax)
#     attention_scores = activator(energies)
#     #calculate a context vector
#     context = dotor([attention_scores,a])

#     return context

### Shared Layer
# n_h = 64 #hidden dimensions for encoder 
# n_s = 128 #hidden dimensions for decoder
# decoder_LSTM_cell = LSTM(n_s, return_state = True) #decoder_LSTM_cell
# output_layer = Dense(vocab_size, activation="softmax") #softmax output layer
# embed_layer = Embedding(len(word_to_idx), n_h, input_length=maxlen,mask_zero=True)
# bi_LSTM = Bidirectional(LSTM(n_h, return_sequences=True))

# out_s1 = Dense(256,activation="relu")
# out_s2 = Dense(256,activation="relu")

# def build_model(Tx, Ty, n_h, n_s, vocab_size, machine_vocab_size):

#     # Define the input of your model
#     X = Input(shape=(Tx,))
#     # Define hidden state and cell state for decoder_LSTM_Cell
#     s0 = Input(shape=(n_s,), name='s0')
#     c0 = Input(shape=(n_s,), name='c0')
#     s = s0
#     c = c0
    
#     # Initialize empty list of outputs
#     outputs = list()

#     #Encoder Bi-LSTM
#     h = embed_layer(X)
# #     h = bi_LSTM(embed)
  
#     #Iterate for Ty steps (Decoding)
#     for t in range(Ty):
    
#         #Perform one step of the attention mechanism to calculate the context vector at timestep t
#         context = one_step_attention(h, s)
       
#         # Feed the context vector to the decoder LSTM cell
#         s, _, c = decoder_LSTM_cell(context,initial_state=[s,c])
           
#         # Pass the decoder hidden output to the output layer (softmax)
#         o1 = out_s1(s)
#         o2 = out_s2(o1)
#         out = output_layer(o2)
        
#         # Append an output list with the current output
#         outputs.append(out)
    
#     #Create model instance
#     model = Model(inputs=[X,s0,c0],outputs=outputs)
    
#     return model

In [10]:
# model = build_model(Tx, Ty, n_h, n_s, vocab_size, vocab_size)
# opt = Adam(lr= 0.005, decay = 0.0001)
# model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['categorical_accuracy'])

# m = 20000
# s0 = np.zeros((m, n_s))
# c0 = np.zeros((m, n_s))
# outputs = list(Y.swapaxes(0,1))

# model.fit([X, s0, c0], outputs, epochs=5, batch_size=64, shuffle=True)

In [11]:
maxlen

140

In [12]:
in_x = maxlen+5
# in_x = 40
n_h = 256
def build_simple_model():
    inp = Input(shape=(in_x,))
    mod = Embedding(len(word_to_idx), n_h, input_length=in_x)(inp)
    mod = Bidirectional(LSTM(n_h, return_sequences=True))(mod)
    mod = BatchNormalization()(mod)
    mod = Dropout(0.2)(mod)
    mod = Flatten()(mod)
    
    mod = Dense(512)(mod)
    mod = BatchNormalization()(mod)
    mod = Activation('relu')(mod)
    mod = Dropout(0.2)(mod)
    
    out = Dense(vocab_size, activation="softmax")(mod)
    
    model = Model(inputs=inp, outputs=out)
#     model.compile(loss="categorical_crossentropy", optimizer=Adam(lr= 0.01, decay = 0.0001), metrics=['categorical_accuracy'])
    return model

model = build_simple_model()

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 145)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 145, 256)          24576     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 145, 512)          1050624   
_________________________________________________________________
batch_normalization_1 (Batch (None, 145, 512)          2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 145, 512)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 74240)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               38011392  
__________

In [14]:
X_train = []
y_train = []
for i in X: ### เพิ่มเลขตัวนี้เท่าที่ Ram รับไหว
    i = [0]*(in_x-1) + i 
    for j in range(in_x,len(i)):
        y_train.append(i[j])
        tmp = []
        for k in range(in_x):
            tmp.append(i[j-in_x+k])
        X_train.append(np.array(tmp))
X_train = np.array(X_train)
y_train = np.array(y_train).reshape(-1,1)
# y_train = to_categorical(y_train,vocab_size)

In [15]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(in_x,), n_channels=1,
                 n_classes=vocab_size, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        list_IDs_y = [self.labels[k] for k in indexes]
        
        # Generate data
        X, y = self.__data_generation(list_IDs_temp, list_IDs_y)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp, list_IDs_y):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim,))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = ID
            
            # Store class
            y[i] = list_IDs_y[i]

        return X, to_categorical(y, self.n_classes)

In [16]:
params = {'dim': (in_x,),
          'batch_size': 1024,
          'n_classes': vocab_size,
          'n_channels': 1,
          'shuffle': True}

In [17]:
training_generator = DataGenerator(X_train, y_train, **params)

In [18]:
model.load_weights("new_char_model_05.h5")

In [37]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001, decay= 0.0000001), metrics=['categorical_accuracy'])

In [19]:
# history = model.fit(X_train, y_train, epochs=5, batch_size=2048, shuffle=True, verbose=1)
# model.fit_generator(generator=training_generator, epochs=5, verbose=1, steps_per_epoch=len(X_train)//params["batch_size"])

In [67]:
# model.save_weights("8_5unk_model.h5")

### Greedy

In [28]:
def texts_to_sequences(text, word_to_index):
    text = text.strip().split()
#     print(text)
    token_list = [word_to_index[x] for x in text]
    return token_list


current_text = "<s> ฝ"
# current_text = "<s> ก"
probs = []
for _ in range(in_x):
    ### YOUR CODE HERE

    tmp = texts_to_sequences(current_text, word_to_idx)

    tmp = pad_sequences([tmp], maxlen=in_x, value=0.0)

    output_word = model.predict(tmp)
    probs.append(output_word.max())
    output_word = np.argmax(output_word)
    output_word = idx_to_word[output_word]

    ### END YOUR CODE
    current_text += " " + output_word

In [29]:
current_text

'<s> ฝ ่ า ย พ ร ะ อ ง ค ์ ท ร ง ศ ร ี ส ุ ว ร ร ณ / เ ห ็ น เ ห ็ น เ ห ็ น เ ห ็ น เ ป ็ น ส ุ ข า ร ณ ์ / จ ึ ง จ ะ ไ ด ้ เ ห ็ น เ ป ็ น เ ป ็ น ท ี ่ ร ั ก / เ ห ็ น จ ะ ไ ด ้ เ ห ็ น เ ป ็ น เ ป ็ น เ ป ็ น ท ี ่ น ี ้ น ี ้ น ี ้ น ี ้ อ ย ม า ย                                            '

### Beam Search

In [30]:
def cal_score(score_list, length, normalized=False):

    seq_score = 0
    tmp_length = min(length,len(score_list))
    for i in range(length):

        seq_score+=np.log(score_list[i] + 0.00000001)
    
    if(normalized):
        seq_score/=length
    
    return np.exp(seq_score)

def beam_search_decode(seed_text, max_gen_len, model, word_to_index, index_to_word, max_sequence_len, beam_size, normalized=False):

    beams = [[seed_text, [], 0]]

    for _ in range(max_gen_len):
        
        tmp_beams = beams.copy()
        beams = []
        for beam in tmp_beams:
            if(beam[0].split()[-1] == "</s>"):
                beams.append(beam)
                continue
                
            tmp = texts_to_sequences(beam[0], word_to_index)
            tmp = pad_sequences([tmp], maxlen=max_sequence_len, value=0.0)
            
            output_word = model.predict(tmp).reshape(-1)
            top_beam_idx = np.argsort(output_word)[-1*beam_size:]
            top_beam_values = [output_word[i] for i in top_beam_idx]
            
            for i in range(beam_size):
                tmp_beam_0 = beam[0] + " " + index_to_word[top_beam_idx[i]]
                tmp_beam_1 = beam[1] + [top_beam_values[i]]
                tmp_beam_2 = beam[2] + 1

                beams.append([tmp_beam_0, tmp_beam_1, tmp_beam_2])
                
        b_cal = []
        for beam in beams:
            b_cal.append(cal_score(beam[1], beam[2], normalized))
        top_beam_idx = np.argsort(b_cal)[-1*beam_size:]
        beams = [beams[x] for x in top_beam_idx]
    
    return beams

In [ ]:
sample_seeds = ["<s> ฝ", "<s> เ",]
# sample_seeds = [ "america"]

for seed in sample_seeds:
    
#     print("-Greedy-")
#     ans_greedy = greedy_decode(seed, 10, model, word_to_index, index_to_word, input_len)
#     tmp_greedy = clean_output(ans_greedy[0],"Eos")
#     print(tmp_greedy, cal_score(ans_greedy[1],min(len(tmp_greedy.split()), len(ans_greedy[0].split())-1), norm))
#     print()
    
#     print("-Unnormalized-")
#     ans = beam_search_decode(seed, 10, model, word_to_index, index_to_word, input_len, 5, normalized=False)
#     top_1 = ans[4]
#     top_2 = ans[3]
#     print(clean_output(top_1[0], 'eos').title(), cal_score(top_1[1], top_1[2], False))  
#     print(clean_output(top_2[0], 'eos').title(), cal_score(top_2[1],top_2[2], False))
#     print()
    
    print("-Normalized-")
    ans = beam_search_decode(seed, in_x, model, word_to_idx, idx_to_word, in_x, 2, normalized=True)
    print(ans[1])
    
    print()
    print("____")
    pass


-Normalized-
['<s> ฝ ่ า ย พ ร ะ อ ง ค ์ ท ร ง ศ ร ี ส ุ ว ร ร ณ ร ั ต น ์ / เ ห ม ื อ น อ ย ่ า ง เ ร ื ่ อ ง ร า ว ก ร ะ ห ว ั ด ห ว ั ง / พ ร ะ ส ั ่ ง เ ส ร ็ จ ส ำ เ ร ็ จ เ ส ร ็ จ จ า ร ย ์ / เ ส ี ย ง ค ร ั ้ น ค ร ั ่ ง พ ร า ห ม ณ ์ พ ร ะ ท ั ย ธ า น ี ่ า ล า ว ่ า น ี                              ', [0.93416196, 0.9980757, 0.98686165, 0.29290777, 0.8553386, 0.9698036, 0.2672103, 0.822596, 0.94945645, 0.9976828, 0.4053514, 0.8485812, 0.98997974, 0.17646337, 0.39683193, 0.63135934, 0.68561435, 0.684838, 0.8893625, 0.9826649, 0.9837446, 0.99355584, 0.153647, 0.8024082, 0.28795826, 0.46122706, 0.9472398, 0.9879588, 0.15510085, 0.20685707, 0.22779466, 0.9677416, 0.9918104, 0.97090155, 0.09490545, 0.26751858, 0.5331597, 0.94628763, 0.694505, 0.12772065, 0.1950263, 0.6563752, 0.19596805, 0.99757427, 0.9339443, 0.16701752, 0.3233393, 0.48177928, 0.11700154, 0.1822179, 0.44190237, 0.2560029, 0.2711085, 0.3845862, 0.43398002, 0.20230475, 0.59899575, 0.36430773, 0.47412091, 0.9116616,